In [1]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

2022-09-22 23:34:04.369609: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
TF_ENABLE_ONEDNN_OPTS=0

## test audio

In [3]:
import numpy as np
import IPython
T = 2.0    # seconds
sr = 22050  # sample rate
t = np.linspace(0, T, int(T*sr), endpoint=False)  # time variable
x = 0.5*np.sin(2*np.pi*440*t)                # pure sine wave at 440 Hz
IPython.display.Audio(x, rate=sr)


## initial

In [5]:
ori_data_path = '/D/TWCC/work/cgh_2022/exp/binear_classification/data/0401_0916/Stoping/adult/'
DATE = "0401_0916"
npz_save_path = '/D/TWCC/work/cgh_2022/exp/binear_classification/npzdata/Stoping-error/'+DATE+'/'
SUBJET = 'adult'
if not os.path.exists(npz_save_path):
    os.makedirs(npz_save_path)

npz_name = 'Stoping_0_1_data'+DATE+'_'+SUBJET

SEED = 42
KFOLD = True
K = 3

## make Dataset

In [6]:
classes=[]

def load_class():
    for _, dirs, _ in os.walk(os.path.abspath(ori_data_path)):
        break
    dirs = [int(x) for x in dirs]
    dirs.sort()
    dirs = [str(x) for x in dirs]
    return dirs

classes=load_class()
print(classes)

['0', '16']


In [8]:
filePaths = []
tmp_label = []
for n in classes:
    ori_path = ori_data_path + str(n)+'/'
    ori_n = 0
    for root2, _, files2 in os.walk(os.path.abspath(ori_path)):
        for file2 in files2:
            if('wav' in file2):
                filePaths.append(os.path.join(root2, file2))
                tmp_label.append(int(n))
                ori_n = ori_n+1
    print('Class: ', n, ' files: ',ori_n)
# random.Random(SEED).shuffle(file_path)  # 把tmp_list弄亂


Class:  0  files:  489
Class:  16  files:  1694


## stat sec

In [10]:
tmp_sec_lists = []
for i, filePath in enumerate(filePaths):
    try:
        y, sr = librosa.load(filePath, sr=16000)
    except:
        print("read error: ", filePath)
    if len(y) <= 0:
        break
    else:
        tmp_sec_lists.append(len(y)/sr)
    

In [13]:
import pandas as pd
s = pd.Series(tmp_sec_lists)
s.describe()


count    2183.000000
mean        1.974201
std         0.225659
min         0.609563
25%         2.031750
50%         2.031750
75%         2.031750
max         2.153688
dtype: float64

In [14]:
wav_data = []
fix_n_sec = 2
mini_sec = 0.5
wav_label = []
for i, filePath in enumerate(filePaths):
    try:
        y, sr = librosa.load(filePath,sr =16000)
    except:
        print("read error: ", filePath)
    if len(y)<=0:
        break;
    if len(y)/sr < mini_sec:
        print('len < '+mini_sec+': ', filePath)
        break;
    if len(y)/sr > fix_n_sec:  # pad if len less 1 sec
        x = y[:fix_n_sec * sr]
    elif mini_sec < len(y)/sr < fix_n_sec :
        z = int(fix_n_sec*sr - len(y))
        if(z%2==0):
            z1 = int(z/2)
            z2 = int(z/2)
        else:
            z1 = int((z-1)/2)
            z2 = int(z1 + 1)
        try:
            x = np.pad(y,(z1,z2), 'linear_ramp', end_values=(0, 0))
        except:
            print("pad error: ", filePath)
    
    wav_data.append(x)
    wav_label.append(tmp_label[i])
X = np.asarray(wav_data)
y = np.asarray(wav_label)


In [16]:
print(f"X.shape: {X.shape}")
print(f"y.shape: {y.shape}")


X.shape: (2183, 32000)
y.shape: (2183,)


In [15]:
y

array([ 0,  0,  0, ..., 16, 16, 16])

In [17]:
from IPython.display import Audio

Audio(X[5], rate=sr)


In [18]:
if (KFOLD):
    # example of stratified k-fold cross-validation with an imbalanced dataset
    from sklearn.datasets import make_classification
    from sklearn.model_selection import StratifiedKFold
    # generate 2 class dataset
    kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED)
    # enumerate the splits and summarize the distributions
    i = 1
    for train_ix, test_ix in kfold.split(X, y):
        # select rows
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # summarize train and test composition
        train_0, train_16 = len(y_train[y_train == 0]), len(y_train[y_train == 16])
        test_0, test_16 = len(y_test[y_test == 0]), len(y_test[y_test == 16])
        print(f"part{i}:")
        print('>Train: 0=%d, 16=%d, Test: 0=%d, 16=%d' %
              (train_0, train_16, test_0, test_16))
        print(f"saving {npz_save_path+npz_name}_part_{i}")
        np.savez(
            npz_save_path+npz_name+"_part_"+str(i),
            X_train=np.asarray(X_train),
            y_train=np.asarray(y_train),
            X_test=np.asarray(X_test),
            y_test=np.asarray(y_test)
        )        
        i = i + 1
else:  
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=SEED, stratify=y)
    classes = np.unique(y_train)
    train_0, train_16 = len(y_train[y_train == 0]), len(y_train[y_train == 16])
    test_0, test_16 = len(y_test[y_test == 0]), len(y_test[y_test == 16])
    print('>Train: 0=%d, 16=%d, Test: 0=%d, 16=%d' %
          (train_0, train_16, test_0, test_16))
    print(f"saving {npz_save_path+npz_name}")
    np.savez(
        npz_save_path+npz_name,
        X_train=np.asarray(X_train),
        y_train=np.asarray(y_train),
        X_test=np.asarray(X_test),
        y_test=np.asarray(y_test)
    )
    

part1:
>Train: 0=326, 16=1129, Test: 0=163, 16=565
saving /D/TWCC/work/cgh_2022/exp/binear_classification/npzdata/Stoping-error/0401_0916/Stoping_0_1_data0401_0916_adult_part_1
part2:
>Train: 0=326, 16=1129, Test: 0=163, 16=565
saving /D/TWCC/work/cgh_2022/exp/binear_classification/npzdata/Stoping-error/0401_0916/Stoping_0_1_data0401_0916_adult_part_2
part3:
>Train: 0=326, 16=1130, Test: 0=163, 16=564
saving /D/TWCC/work/cgh_2022/exp/binear_classification/npzdata/Stoping-error/0401_0916/Stoping_0_1_data0401_0916_adult_part_3


## mix

In [19]:
list_a = ['1','2','3']
for i in list_a:
    ADULTPATH = '/D/TWCC/work/cgh_2022/exp/binear_classification/npzdata/Stoping-error/0401_0916/Stoping_0_1_data0401_0916_adult_part_'+i+'.npz'
    KIDPATH = '/D/TWCC/work/cgh_2022/exp/binear_classification/npzdata/Stoping-error/0401_0916/Stoping_0_1_data0401_0916_child_part_'+i+'.npz'
    x_adult = np.load(ADULTPATH, mmap_mode='r', allow_pickle=True)
    Xadult_train = x_adult['X_train']
    yadult_train = x_adult['y_train']
    Xadult_test = x_adult['X_test']
    yadult_test = x_adult['y_test']
    x_child = np.load(KIDPATH, mmap_mode='r', allow_pickle=True)
    Xchild_train = x_child['X_train']
    ychild_train = x_child['y_train']
    Xchild_test = x_child['X_test']
    ychild_test = x_child['y_test']
    X_train = np.concatenate((Xadult_train, Xchild_train, Xadult_test), axis=0)
    y_train = np.concatenate((yadult_train, ychild_train, yadult_test), axis=0)
    X_test = Xchild_test
    y_test = ychild_test
    npz_name = 'Backing_0_1_data'+DATE+'_'+'mix'
    np.savez(
        npz_save_path+npz_name+"_part_"+str(i),
        X_train=np.asarray(X_train),
        y_train=np.asarray(y_train),
        X_test=np.asarray(X_test),
        y_test=np.asarray(y_test)
    )
